In [11]:
!pip install transformers
# !pip install transformers==2.8.0
!git clone https://github.com/sitingGZ/bert-sner.git #&& cd bert-sner
!pip install pytorch_lightning
%cd /content/bert-sner/modules # change to the modules directory

fatal: destination path 'bert-sner' already exists and is not an empty directory.
/content/bert-sner/modules


In [18]:
# some times needs this second execution; often would show no such directory, but in fact it works
%cd /content/bert-sner/modules # change to the modules directory

[Errno 2] No such file or directory: '/content/bert-sner/modules # change to the modules directory'
/content/bert-sner/modules


In [16]:
# check that the location is /content/bert-sner/modules
!pwd

/content/bert-sner/modules


In [19]:
import warnings

# mount the google drive
from google.colab import drive
drive.mount('/content/gdrive')

warnings.filterwarnings('ignore')

Mounted at /content/gdrive


In [20]:
# import sys
# sys.path.append('modules') # seemed not to work, used %cd /content/bert-sner/modules above

import torch
from transformers import AutoConfig, AutoTokenizer, AutoModelForMaskedLM, EncoderDecoderConfig
from BERT2span_semantic_disam import BERT2span
from helpers import load_config, set_seed
from inference import final_label_results_rescaled

base_name =  "bert-base-german-cased"
configs = load_config('../configs/step3_gpu_span_semantic_group.yaml') # path is adjusted from inside of the modules dir
tokenizer = AutoTokenizer.from_pretrained(base_name)
bertMLM = AutoModelForMaskedLM.from_pretrained(base_name)
bert_sner = BERT2span(configs, bertMLM, tokenizer)

checkpoint_path = "../../gdrive/MyDrive/ner-models/german_bert_ex4cds_500_semantic_term.ckpt" # checkpoints from https://huggingface.co/sitingGZ/german-bert-clinical-ner/tree/main
# checkpoint_path = "../../gdrive/MyDrive/ner-models/german_bert_muchmore_semantic_term.ckpt" # put to the google drive
state_dict = torch.load(checkpoint_path, map_location=torch.device('cuda')) # cpu or cuda
bert_sner.load_state_dict(state_dict, strict=False) # original implementation triggered an error, strict=False fixes it
bert_sner.eval()

suggested_terms = {'Condition': 'Zeichen oder Symptom',
                   'DiagLab': 'Diagnostisch und Laborverfahren',
                   'LabValues': 'Klinisches Attribut',
                   'HealthState': 'Gesunder Zustand',
                   'Measure': 'Quantitatives Konzept',
                   'Medication': 'Pharmakologische Substanz',
                   'Process': 'Physiologische Funktion',
                   'TimeInfo': 'Zeitliches Konzept'}

words = "Welche Erreger verursachen häufig eine frühe postoperative Osteitis oder periimplantäre Infektion?".split() # text to be tagged
words_list = [words]
heatmaps, ner_results = final_label_results_rescaled(words_list, tokenizer, bert_sner, suggested_terms, threshold=0.5) # thresholds


Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
# first model threshold 0.9
ner_results

[[('Welche', 0, 'O'),
  ('Erreger', 0, 'O'),
  ('verursachen', 5, 'Measure'),
  ('häufig', 0, 'O'),
  ('eine', 0, 'O'),
  ('frühe', 2, 'DiagLab'),
  ('postoperative', 0, 'O'),
  ('Osteitis', 1, 'Condition'),
  ('oder', 0, 'O'),
  ('periimplantäre', 3, 'LabValues'),
  ('Infektion?', 1, 'Condition')]]

In [9]:
# first model threshold 0.5
ner_results

[[('Welche', 0, 'O'),
  ('Erreger', 3, 'LabValues'),
  ('verursachen', 5, 'Measure'),
  ('häufig', 5, 'Measure'),
  ('eine', 0, 'O'),
  ('frühe', 2, 'DiagLab'),
  ('postoperative', 6, 'Medication'),
  ('Osteitis', 1, 'Condition'),
  ('oder', 0, 'O'),
  ('periimplantäre', 3, 'LabValues'),
  ('Infektion?', 1, 'Condition')]]

In [27]:
# second model threshold 0.9
ner_results

[[('Welche', 0, 'O'),
  ('Erreger', 3, 'LabValues'),
  ('verursachen', 5, 'Measure'),
  ('häufig', 5, 'Measure'),
  ('eine', 0, 'O'),
  ('frühe', 2, 'DiagLab'),
  ('postoperative', 6, 'Medication'),
  ('Osteitis', 1, 'Condition'),
  ('oder', 0, 'O'),
  ('periimplantäre', 3, 'LabValues'),
  ('Infektion?', 1, 'Condition')]]

In [19]:
# second model threshold 0.5
ner_results

[[('Welche', 0, 'O'),
  ('Erreger', 0, 'O'),
  ('verursachen', 0, 'O'),
  ('häufig', 0, 'O'),
  ('eine', 0, 'O'),
  ('frühe', 8, 'TimeInfo'),
  ('postoperative', 4, 'HealthState'),
  ('Osteitis', 1, 'Condition'),
  ('oder', 0, 'O'),
  ('periimplantäre', 7, 'Process'),
  ('Infektion?', 2, 'DiagLab')]]